# Sketch of handling credits in Oscar/Julia


We want to give proper credit to other software used. To this end we introduce a `Credits` struct that retains the information of the credits we want to display.

- Printing of the credits can be enabled on demand, by setting a global variable. If this variable does not exist, nothing is printed.
- Credits are only printed once.
- Another macro will offer the possibility to collect all credits in a specific computation.

I suspect that not every dev is happy if we suddenly spew credits everywhere. Nevertheless, if later in the refereeing Oscar code appears, referees can use this approach to find missing references.

In [1]:
using MacroTools

In [2]:
mutable struct MutableBool
    x::Bool
end

struct Credits
    name::String
    bibtex::String
    displayed::MutableBool
end

Credits(x,y) = Credits(x,y, MutableBool(false))

Credits

With the `MutableBool`, we can now check whether the credits were already displayed or not, and only diplay them once.

In [3]:
function display_credits(cr::Credits, func::String)
    if @isdefined print_credits
        if print_credits
            if !cr.displayed.x
                cr.displayed.x = true
                println("Using $(cr.name)")
                println("in function $(func)")
            end
        end
    end
end

display_credits (generic function with 1 method)

In [4]:
creditsA = Credits("credits of A", "bibtex of credits of A")
creditsB = Credits("credits of B", "bibtex of credits of B")

Credits("credits of B", "bibtex of credits of B", MutableBool(false))

In [5]:
display_credits(creditsA, "ggg")

Then we can attach the credits to functions with a macro. The following is a modified copy of the `@attr` macro of `AbstractAlgebra.jl`.

In [6]:
macro annotate_credits(ex1, exs...)
    # TODO: Is there some nicer way to pass on the Credits object?
    credits = eval(ex1)
    isa(credits, Credits) || error("First argument must be credits.")

    if length(exs) == 0
        throw(ArgumentError("Too few macro arguments"))
    else
        expr = exs[1]
    end
    d = MacroTools.splitdef(expr)

    # store the original function name
    name = d[:name]

    # take the original function and rename it; use a unique name to ensure
    # there are no clashes caused by spurious additional methods
    compute_name = gensym("compute_$(name)")
    compute_def = copy(d)
    compute_def[:name] = compute_name
    compute = MacroTools.combinedef(compute_def)

    argname = d[:args][1]
    wrapper_def = copy(d)
    wrapper_def[:name] = name
    wrapper_def[:body] = quote
        # TODO: Do not pass string here, but symbol instead?
        display_credits($credits, string($name))
        return $(compute_name)($argname)
    end
    # insert the correct line number, so that `functionloc(name)` works correctly
    wrapper_def[:body].args[1] = __source__
    wrapper = MacroTools.combinedef(wrapper_def)

    result = quote
        $(compute)
        Base.@__doc__ $(wrapper)
    end

    return esc(result)
end


@annotate_credits (macro with 1 method)

In [7]:
@annotate_credits creditsA function g(x::Int)
    2*x
end

@annotate_credits creditsB function h(x::Int)
    5*x
end

h (generic function with 1 method)

In [8]:
g(2)

4

In [9]:
print_credits = true
g(3)

Using credits of A
in function g


6

# Collecting credits

This can be achieved by manipulating the `display_credits` method.

In [10]:
function display_credits(cr::Credits, func::String)
    if @isdefined print_credits
        if print_credits
            if !cr.displayed.x
                cr.displayed.x = true
                println("Using $(cr.name)")
                println("in function $(func)")
            end
        end
    end
    if @isdefined CollectedCredits
        if isa(CollectedCredits, Dict{Credits, Set{String}})
            if !haskey(CollectedCredits, cr)
                CollectedCredits[cr] = Set{String}([func])
            else
                push!(CollectedCredits[cr], func)
            end
        end
    end
end

display_credits (generic function with 1 method)

In [11]:
CollectedCredits = Dict{Credits, Set{String}}()

Dict{Credits, Set{String}}()

In [12]:
g(3)
h(7)

Using credits of B
in function h


35

In [13]:
CollectedCredits

Dict{Credits, Set{String}} with 2 entries:
  Credits("credits of B", "bibtex of credits of B", MutableBool(t… => Set(["h"])
  Credits("credits of A", "bibtex of credits of A", MutableBool(t… => Set(["g"])

The advantage is that the user can even track the location of the usage of the external software.